### Student Information
Name:  李博業

Student ID: 410978058

GitHub ID: BORYA-dev

Kaggle name:

Kaggle private scoreboard snapshot:

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home exercises** in the [DM2024-Lab2-master Repo](https://github.com/didiersalazar/DM2024-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook.


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework) regarding Emotion Recognition on Twitter by this link: https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework. The scoring will be given according to your place in the Private Leaderboard ranking:
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (0.6N + 1 - x) / (0.6N) * 10 + 20 points, where N is the total number of participants, and x is your rank. (ie. If there are 100 participants and you rank 3rd your score will be (0.6 * 100 + 1 - 3) / (0.6 * 100) * 10 + 20 = 29.67% out of 30%.)   
    Submit your last submission **BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)**. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developing the model for the competition (You can use code and comment on it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained.


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)__.

- emotion
  - The emotion(answer) of data
    - sadness
    - disgust
    - anticipation
    - joy
    - trust
    - anger
    - fear
    - surprise

- data_identification :
  - identify the id is train or test
  
- tweets_DM.json : Main Data
  - type : list
  - length = 1867535
  - record :
    {
    '_score': 391,
    '_index': 'hashtag_tweets',
    '_source': {'tweet': {'hashtags': ['Snapchat'],
      'tweet_id': '0x376b20',
      'text': 'People who post "add me on #Snapchat" must be dehydrated. Cuz man.... that\'s <LH>'}},
    '_crawldate': '2015-05-23 11:42:47',
    '_type': 'tweets'
    }

In [31]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split

import nltk
import re
import stringprep
nltk.download('punkt_tab')
# nltk.download('wordnet')

import pickle

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer


from sklearn.metrics import accuracy_score

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


- Prepare Data

In [3]:
### Begin Assignment Here

### 準備資料

file_path = '/content/drive/MyDrive/Data_Lab2_HW/tweets_DM.json'

data = []

with open(file_path, 'r', encoding='utf-8') as file:
    for line_number, line in enumerate(file, start=1):
        try:
            data.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"Error in line {line_number}: {e}")


emotion_data = pd.read_csv('/content/drive/MyDrive/Data_Lab2_HW/emotion.csv')
data_id = pd.read_csv('/content/drive/MyDrive/Data_Lab2_HW/data_identification.csv')

# 生成DataFrame
temp = []
for sub_d in data:
  temp.append(sub_d['_source']['tweet'])
data_df = pd.DataFrame(temp)

# 排序left join
data_df.sort_values("tweet_id",ignore_index = True, inplace = True)
emotion_data.sort_values("tweet_id",ignore_index = True, inplace = True)
data_id.sort_values("tweet_id",ignore_index = True, inplace = True)


# left_join raw data and emotion_data
data_df = pd.merge(left=data_df, right=emotion_data, how="left", on ="tweet_id")
data_df = pd.merge(left=data_df, right=data_id, how="left", on ="tweet_id")

data_df.head(5)

In [ ]:
# 訓練集、測試集
training_data = data_df.query("identification == 'train'").reset_index(drop =True)
testing_data = data_df.query("identification == 'test'").reset_index(drop =True)

In [ ]:
# 訓練集抽樣

train_data_sample = training_data.sample(frac=0.3, random_state=20)
train_data_sample = train_data_sample.reset_index(drop=True)
x_train = train_data_sample.drop(columns = ["emotion", "identification"])
y_train = train_data_sample.emotion

# 分層抽樣
# Strafied sampling
x_train, x_test, y_train, y_test = train_test_split(
    x_train, y_train, test_size=0.2, random_state=20, stratify=y_train
)

x_train.reset_index(drop=True, inplace = True)
y_train.reset_index(drop=True, inplace = True)

- Pre-processing

In [6]:
emoji_dict = {
    '😂': '[joy]',
    '❤️': '[love]',
    '😍': '[adoration]',
    '😭': '[cry]',
    '❤': '[care]',
    '😊': '[happy]',
    '🙏': '[pray]',
    '😘': '[kiss]',
    '💕': '[love_each_other]',
    '🔥': '[fire]',
    '😩': '[weary]',
    '🤔': '[think]',
    '💯': '[perfect]',
    '💙': '[loyalty]',
    '🙄': '[annoyed]',
    '😁': '[happy]',
    '🙌': '[celebrate]',
    '🙏🏾': '[pray]',
    '👍': '[approve]',
    '🙏🏽': '[pray]',
    '♡':'[love]',
    '🕘':'[time]',
    '🙋':'[hi]'
}

In [27]:
def cleaning(text):
    text = re.sub(r'[@#$%^&*()_+{}[\]:;"\'<>,.~\\/\-|=]', '', text)

    # remove some common words
    text = re.sub(r'\b(?:and|is|in|the|on|of|to|for)\b', '', text, flags=re.IGNORECASE)
    # remove numbers
    text = re.sub(r'\b\d+\b', '', text)
    # remove blank space
    text = re.sub(r'\s+', ' ', text).strip() # \s+ 一個或多個空白字符
    # remove links
    text = re.sub('((www.[^s]+)|(https?://[^s]+))',' ',text)
    return text


# emoji
def trans_emoji(text):
  for key, value in emoji_dict.items():
    text = text.replace(key, value)
  return text

# Stemming
# st = nltk.PorterStemmer()

# def stemming_on_text(data):
#     '''
#     input should be tokenized words : list
#     '''
#     text = [st.stem(word) for word in data]
#     return text

# # Lemmatization
# lm = nltk.WordNetLemmatizer()
# def lemmatizer_on_text(data):
#     text = [lm.lemmatize(word) for word in data]
#     return text


def pre_processing(data):

  '''
  data:DataFrame with text
  '''

  dataset = data['text']

  # 轉小寫
  dataset = dataset.apply(lambda text: text.lower().strip())

  # 移除 removing
  dataset = dataset.apply(lambda x: x.replace("<lh>",""))
  dataset = dataset.apply(lambda x: x.replace("<LH>",""))
  dataset = dataset.apply(lambda x: x.replace("#",""))

  # change emoji
  dataset = dataset.apply(lambda x: trans_emoji(x) )

  # cleaning some words
  dataset = dataset.apply(lambda x: cleaning(x) )

  dataset = dataset.apply(lambda x: x.replace("#",""))

  dataset.name = "pre_processing_text"

  tt = dataset.apply(lambda x: nltk.word_tokenize(x))
  tt.name = "tokens"

  pre_df = pd.concat([dataset,pd.DataFrame(tt)], axis=1)

  return pre_df

In [28]:
# Combine tje columns with pro-processing & original text

x_train = pd.concat([x_train, pre_processing(x_train)], axis=1)
x_train.reset_index(drop=True, inplace=True)

x_test = pd.concat([x_test, pre_processing(x_test)], axis=1)
x_test.reset_index(drop=True, inplace=True)

testing_data = pd.concat([testing_data, pre_processing(testing_data)], axis=1)
testing_data.reset_index(drop=True, inplace=True)

In [29]:
x_train.head(5)

,hashtags,tweet_id,text,pre_processing_text,tokens
0,[],0x2d34ef,ppl on my snap really be in a different state ...,ppl my snap really be a different state rn tra...,"[ppl, my, snap, really, be, a, different, stat..."
1,[baduy],0x27c05d,@aam_offshore @FSTopacio Haha just shows how u...,aamoffshore fstopacio haha just shows how unpr...,"[aamoffshore, fstopacio, haha, just, shows, ho..."
2,"[notetaking, tips]",0x33cde7,Any tips on becoming a better notetaker? Askin...,any tips becoming a better notetaker? asking a...,"[any, tips, becoming, a, better, notetaker, ?,..."
3,"[either, feeling, nations]",0x298606,And I am not <LH> #either <LH> #feeling <LH> ...,i am not either feeling nations,"[i, am, not, either, feeling, nations]"
4,[coffee],0x340b85,I am so excited!!! Had a wonderful surprise fr...,i am so excited!!! had a wonderful surprise fr...,"[i, am, so, excited, !, !, !, had, a, wonderfu..."


- TFIDF

In [32]:
# tweet_tokenizer = nltk.tokenize.TweetTokenizer() # 本來有用 max_df=0.9, tokenizer = tweet_tokenizer.tokenize
TF_IDF_vectorizer = TfidfVectorizer(max_features=1100) # 3000 2500 2000 都會爆
output_vectorizer = TF_IDF_vectorizer.fit_transform(x_train.text.values)  # fit + transform

In [17]:
output_vectorizer_test = TF_IDF_vectorizer.transform(x_test.text.values)  # only transform

In [33]:
output_vectorizer_true_test = TF_IDF_vectorizer.transform(testing_data.text.values)  # only transform

In [42]:
# 寫出

#訓練+測試
with open('/content/drive/MyDrive/Data_Lab2_HW/tf-idf-trained.pkl', 'wb') as file:  # 'wb' 表示以二進位寫入模式
    pickle.dump([TF_IDF_vectorizer, output_vectorizer, y_train, output_vectorizer_test, y_test], file)

# 真正測試資料
with open('/content/drive/MyDrive/Data_Lab2_HW/testing_data.pkl', 'wb') as file:  # 'wb' 表示以二進位寫入模式
    pickle.dump([testing_data, output_vectorizer_true_test], file)


In [2]:
#讀入

with open('/content/drive/MyDrive/Data_Lab2_HW/tf-idf-trained.pkl', 'rb') as file:  # 'rb' 表示以二進位讀取模式
    tfidf_list = pickle.load(file)

with open('/content/drive/MyDrive/Data_Lab2_HW/testing_data.pkl', 'rb') as file:  # 'rb' 表示以二進位讀取模式
    testing_list = pickle.load(file)

In [44]:
X = tfidf_list[1]
y_train = tfidf_list[2].values

In [34]:
model = RandomForestClassifier()

## training!
DT_model = model.fit(output_vectorizer, y_train)

## predict! training

In [6]:
from joblib import dump, load

In [7]:
# 寫出模型
dump(DT_model, '/content/drive/MyDrive/Data_Lab2_HW/random_forest_3000.joblib')  # 儲存模型

['/content/drive/MyDrive/Data_Lab2_HW/random_forest_3000.joblib']

In [ ]:
# 讀入模型
DT_model = load('/content/drive/MyDrive/Data_Lab2_HW/random_forest.joblib')

In [47]:
y_train_pred = DT_model.predict(X)

In [48]:
## Training Accuracy

acc_train = accuracy_score(y_true=y_train, y_pred=y_train_pred)

print('training accuracy: {}'.format(round(acc_train, 2)))

training accuracy: 0.98


In [49]:
X = tfidf_list[3]

In [50]:
X.shape

(87334, 3000)

In [51]:
# Mini-Testing
# output_vectorizer_testing = TF_IDF_vectorizer.fit_transform(mini_X_test['preprocess_text'])

# Predict Mini-Testing (by Random Forest)
y_test_result = DT_model.predict(X)

acc_train = accuracy_score(y_true=tfidf_list[4].values, y_pred=y_test_result)

print('Mini-testing accuracy: {}'.format(round(acc_train, 2)))

Mini-testing accuracy: 0.5


In [ ]:
DT_model = load('/content/drive/MyDrive/Data_Lab2_HW/random_forest.joblib')

In [12]:
testing_data = testing_list[0]
testing_data.head(5)

,tweet_id,preprocess_text
0,0x1c7f0f,jzed while inappropri af he like wasnt kid
1,0x1c7f12,i tri figur out whi you mean so much me i coul...
2,0x1c7f13,onli “ big plan ” you ever had your life wa pr...
3,0x1c7f17,look back situat old new recent or whateverpeo...
4,0x1c7f18,jasoninthehous whi do you insist talk about cl...


In [13]:
X = testing_list[1]

In [35]:
output_vectorizer_true_test.shape

(411972, 1100)

In [36]:
y_test_result = DT_model.predict( output_vectorizer_true_test )

In [37]:
# 寫出結果
result = pd.DataFrame( {
   "id" : testing_data.tweet_id,
   "emotion" : y_test_result
} )
result.to_csv("/content/drive/MyDrive/submission_test_v8.csv", index = 0)
result.head(5)

,id,emotion
0,0x1c7f0f,joy
1,0x1c7f12,joy
2,0x1c7f13,joy
3,0x1c7f17,joy
4,0x1c7f18,disgust


- Word2Vec

In [ ]:
train_corpus = x_train.tokens.values

In [ ]:
# Training Word2Vec

from gensim.models import Word2Vec

# setting
vector_dim = 100
window_size = 5
min_count = 1
training_epochs = 20

word2vec_model = Word2Vec(sentences=train_corpus,
              vector_size = vector_dim,
              window=window_size,
              min_count=min_count,
              epochs=training_epochs)

In [ ]:
def mean_word2vec(text_list):
  result = np.zeros(vector_dim)
  length = len(text_list)
  for i in text_list:
    try:
      result = result + word2vec_model.wv[i]
    except KeyError as e:
      next
  result = result / length
  return result

In [ ]:
x_train["word2vec_100"] = x_train.apply(lambda x: mean_word2vec(x['tokens']), axis = 1)

In [ ]:
# Training Random Forest

X = list(x_train.word2vec_100.values)

model = RandomForestClassifier()

## training!
DT_model = model.fit(X, y_train.values)

In [ ]:
## Training Accuracy
from sklearn.metrics import accuracy_score

y_train_pred = DT_model.predict(X)

acc_train = accuracy_score(y_true=y_train, y_pred=y_train_pred)

print('training accuracy: {}'.format(round(acc_train, 2)))

In [ ]:
tokens_testing = testing_data.apply(lambda x: tok.tokenize(x["preprocess_text"]), axis = 1).tolist()
testing_data["tokens"] = tokens_testing

In [ ]:
testing_data["word2vec_100"] = testing_data.apply(lambda x: mean_word2vec(x['tokens']), axis = 1)

In [ ]:
# Prediction
X = list(testing_data.word2vec_100.values)

y_test_result = DT_model.predict(X)

result = pd.DataFrame( {
   "id" : testing_data.tweet_id,
   "emotion" : y_test_result
} )
result.to_csv("/content/drive/MyDrive/submission_test_v4.csv", index = 0)
result.head(5)

結果紀錄
- Proprocessing + TFIDF(max=0.9, 3000, tette.tokenizer) + RF 0.36895
- TFIDF(3000) + RF 0.35301
- Preprocessing + Word2Vec + RF 0.34046
- TFIDF(1100) + RF 0.34696
- Word2Vec + RF